Deep Learning
=============

Assignment 4
------------

Previously in `2_fullyconnected.ipynb` and `3_regularization.ipynb`, we trained fully connected networks to classify [notMNIST](http://yaroslavvb.blogspot.com/2011/09/notmnist-dataset.html) characters.

The goal of this assignment is make the neural network convolutional.

In [11]:
# These are all the modules we'll be using later. Make sure you can import them
# before proceeding further.
from __future__ import print_function
import numpy as np
import tensorflow as tf
from six.moves import cPickle as pickle
from six.moves import range

In [12]:
pickle_file = 'notMNIST.pickle'

with open(pickle_file, 'rb') as f:
  save = pickle.load(f)
  train_dataset = save['train_dataset']
  train_labels = save['train_labels']
  valid_dataset = save['valid_dataset']
  valid_labels = save['valid_labels']
  test_dataset = save['test_dataset']
  test_labels = save['test_labels']
  del save  # hint to help gc free up memory
  print('Training set', train_dataset.shape, train_labels.shape)
  print('Validation set', valid_dataset.shape, valid_labels.shape)
  print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 28, 28) (200000,)
Validation set (10000, 28, 28) (10000,)
Test set (18724, 28, 28) (18724,)


Reformat into a TensorFlow-friendly shape:
- convolutions need the image data formatted as a cube (width by height by #channels)
- labels as float 1-hot encodings.

In [13]:
image_size = 28
num_labels = 10
num_channels = 1 # grayscale

import numpy as np

def reformat(dataset, labels):
  dataset = dataset.reshape(
    (-1, image_size, image_size, num_channels)).astype(np.float32)
  labels = (np.arange(num_labels) == labels[:,None]).astype(np.float32)
  return dataset, labels
train_dataset, train_labels = reformat(train_dataset, train_labels)
valid_dataset, valid_labels = reformat(valid_dataset, valid_labels)
test_dataset, test_labels = reformat(test_dataset, test_labels)
print('Training set', train_dataset.shape, train_labels.shape)
print('Validation set', valid_dataset.shape, valid_labels.shape)
print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 28, 28, 1) (200000, 10)
Validation set (10000, 28, 28, 1) (10000, 10)
Test set (18724, 28, 28, 1) (18724, 10)


In [14]:
def accuracy(predictions, labels):
  return (100.0 * np.sum(np.argmax(predictions, 1) == np.argmax(labels, 1))
          / predictions.shape[0])

Let's build a small network with two convolutional layers, followed by one fully connected layer. Convolutional networks are more expensive computationally, so we'll limit its depth and number of fully connected nodes.

In [15]:
batch_size = 16
patch_size = 5
depth = 16
num_hidden = 64

graph = tf.Graph()

with graph.as_default():

  # Input data.
  tf_train_dataset = tf.placeholder(
    tf.float32, shape=(batch_size, image_size, image_size, num_channels))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Variables.
  layer1_weights = tf.Variable(tf.truncated_normal(
      [patch_size, patch_size, num_channels, depth], stddev=0.1))
  layer1_biases = tf.Variable(tf.zeros([depth]))
  layer2_weights = tf.Variable(tf.truncated_normal(
      [patch_size, patch_size, depth, depth], stddev=0.1))
  layer2_biases = tf.Variable(tf.constant(1.0, shape=[depth]))
  layer3_weights = tf.Variable(tf.truncated_normal(
      [image_size // 4 * image_size // 4 * depth, num_hidden], stddev=0.1))
  layer3_biases = tf.Variable(tf.constant(1.0, shape=[num_hidden]))
  layer4_weights = tf.Variable(tf.truncated_normal(
      [num_hidden, num_labels], stddev=0.1))
  layer4_biases = tf.Variable(tf.constant(1.0, shape=[num_labels]))
  
  # Model.
  def model(data):
    conv = tf.nn.conv2d(data, layer1_weights, [1, 2, 2, 1], padding='SAME')
    hidden = tf.nn.relu(conv + layer1_biases)
    conv = tf.nn.conv2d(hidden, layer2_weights, [1, 2, 2, 1], padding='SAME')
    hidden = tf.nn.relu(conv + layer2_biases)
    shape = hidden.get_shape().as_list()
    reshape = tf.reshape(hidden, [shape[0], shape[1] * shape[2] * shape[3]])
    hidden = tf.nn.relu(tf.matmul(reshape, layer3_weights) + layer3_biases)
    return tf.matmul(hidden, layer4_weights) + layer4_biases
  
  # Training computation.
  logits = model(tf_train_dataset)
  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels, logits=logits))
    
  # Optimizer.
  optimizer = tf.train.GradientDescentOptimizer(0.05).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits)
  valid_prediction = tf.nn.softmax(model(tf_valid_dataset))
  test_prediction = tf.nn.softmax(model(tf_test_dataset))

In [16]:
num_steps = 1001

with tf.Session(graph=graph) as session:
  tf.global_variables_initializer().run()
  print('Initialized')
  for step in range(num_steps):
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    batch_data = train_dataset[offset:(offset + batch_size), :, :, :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 50 == 0):
      print('Minibatch loss at step %d: %f' % (step, l))
      print('Minibatch accuracy: %.1f%%' % accuracy(predictions, batch_labels))
      print('Validation accuracy: %.1f%%' % accuracy(
        valid_prediction.eval(), valid_labels))
  print('Test accuracy: %.1f%%' % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 2.819145
Minibatch accuracy: 12.5%
Validation accuracy: 10.0%
Minibatch loss at step 50: 2.218373
Minibatch accuracy: 12.5%
Validation accuracy: 39.5%
Minibatch loss at step 100: 0.993816
Minibatch accuracy: 75.0%
Validation accuracy: 66.0%
Minibatch loss at step 150: 1.223631
Minibatch accuracy: 50.0%
Validation accuracy: 71.8%
Minibatch loss at step 200: 1.033192
Minibatch accuracy: 81.2%
Validation accuracy: 75.6%
Minibatch loss at step 250: 1.195467
Minibatch accuracy: 75.0%
Validation accuracy: 76.5%
Minibatch loss at step 300: 1.031566
Minibatch accuracy: 68.8%
Validation accuracy: 77.8%
Minibatch loss at step 350: 1.043851
Minibatch accuracy: 68.8%
Validation accuracy: 77.2%
Minibatch loss at step 400: 0.742892
Minibatch accuracy: 81.2%
Validation accuracy: 80.8%
Minibatch loss at step 450: 0.822016
Minibatch accuracy: 62.5%
Validation accuracy: 76.4%
Minibatch loss at step 500: 0.867548
Minibatch accuracy: 75.0%
Validation accuracy: 80.1%
M

---
Problem 1
---------

The convolutional model above uses convolutions with stride 2 to reduce the dimensionality. Replace the strides by a max pooling operation (`nn.max_pool()`) of stride 2 and kernel size 2.

---

In [7]:
batch_size = 16
patch_size = 5
depth = 16
num_hidden = 64

graph = tf.Graph()

with graph.as_default():

  # Input data.
  tf_train_dataset = tf.placeholder(
    tf.float32, shape=(batch_size, image_size, image_size, num_channels))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Variables.
  layer1_weights = tf.Variable(tf.truncated_normal(
      [patch_size, patch_size, num_channels, depth], stddev=0.1))
  layer1_biases = tf.Variable(tf.zeros([depth]))
  layer2_weights = tf.Variable(tf.truncated_normal(
      [patch_size, patch_size, depth, depth], stddev=0.1))
  layer2_biases = tf.Variable(tf.constant(1.0, shape=[depth]))
  layer3_weights = tf.Variable(tf.truncated_normal(
      [image_size // 4 * image_size // 4 * depth, num_hidden], stddev=0.1))
  layer3_biases = tf.Variable(tf.constant(1.0, shape=[num_hidden]))
  layer4_weights = tf.Variable(tf.truncated_normal(
      [num_hidden, num_labels], stddev=0.1))
  layer4_biases = tf.Variable(tf.constant(1.0, shape=[num_labels]))
  
  # Model.
  def model(data):
    conv = tf.nn.conv2d(data, layer1_weights, [1, 1, 1, 1], padding='SAME')
    pool1 = tf.nn.max_pool(conv, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='SAME', name='pool1')
    hidden = tf.nn.relu(pool1 + layer1_biases)
    conv = tf.nn.conv2d(hidden, layer2_weights, [1, 1, 1, 1], padding='SAME')
    pool2 = tf.nn.max_pool(conv, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='SAME', name='pool2')
    hidden = tf.nn.relu(pool2 + layer2_biases)
    shape = hidden.get_shape().as_list()
    reshape = tf.reshape(hidden, [shape[0], shape[1] * shape[2] * shape[3]])
    hidden = tf.nn.relu(tf.matmul(reshape, layer3_weights) + layer3_biases)
    return tf.matmul(hidden, layer4_weights) + layer4_biases
  
  # Training computation.
  logits = model(tf_train_dataset)
  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels, logits=logits))
    
  # Optimizer.
  optimizer = tf.train.GradientDescentOptimizer(0.05).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits)
  valid_prediction = tf.nn.softmax(model(tf_valid_dataset))
  test_prediction = tf.nn.softmax(model(tf_test_dataset))

In [8]:
num_steps = 1001

with tf.Session(graph=graph) as session:
  tf.global_variables_initializer().run()
  print('Initialized')
  for step in range(num_steps):
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    batch_data = train_dataset[offset:(offset + batch_size), :, :, :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 50 == 0):
      print('Minibatch loss at step %d: %f' % (step, l))
      print('Minibatch accuracy: %.1f%%' % accuracy(predictions, batch_labels))
      print('Validation accuracy: %.1f%%' % accuracy(
        valid_prediction.eval(), valid_labels))
  print('Test accuracy: %.1f%%' % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 2.823458
Minibatch accuracy: 12.5%
Validation accuracy: 10.4%
Minibatch loss at step 50: 1.892403
Minibatch accuracy: 43.8%
Validation accuracy: 30.7%
Minibatch loss at step 100: 1.059053
Minibatch accuracy: 62.5%
Validation accuracy: 59.4%
Minibatch loss at step 150: 1.327440
Minibatch accuracy: 43.8%
Validation accuracy: 63.0%
Minibatch loss at step 200: 1.324755
Minibatch accuracy: 75.0%
Validation accuracy: 71.8%
Minibatch loss at step 250: 0.915808
Minibatch accuracy: 75.0%
Validation accuracy: 77.2%
Minibatch loss at step 300: 1.004247
Minibatch accuracy: 68.8%
Validation accuracy: 77.0%
Minibatch loss at step 350: 0.940937
Minibatch accuracy: 87.5%
Validation accuracy: 71.4%
Minibatch loss at step 400: 0.786775
Minibatch accuracy: 75.0%
Validation accuracy: 80.3%
Minibatch loss at step 450: 0.754588
Minibatch accuracy: 75.0%
Validation accuracy: 75.5%
Minibatch loss at step 500: 0.584140
Minibatch accuracy: 81.2%
Validation accuracy: 80.8%
M

---
Problem 2
---------

Try to get the best performance you can using a convolutional net. Look for example at the classic [LeNet5](http://yann.lecun.com/exdb/lenet/) architecture, adding Dropout, and/or adding learning rate decay.

---

In [17]:
batch_size = 16
patch_size = 5
depth_six = 6
depth = 16
num_hidden_c5 = 120
num_hidden = 64

graph = tf.Graph()

with graph.as_default():
  # Input data.
  tf_train_dataset = tf.placeholder(tf.float32, shape=(batch_size, image_size, image_size, num_channels))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Placeholder to control dropout probability.
  keep_prob = tf.placeholder(tf.float32)
  
  # Variables.
  # C1: kernel [5, 5, 1, 1]
  layer1_weights = tf.Variable(tf.truncated_normal([patch_size, patch_size, num_channels, num_channels], stddev=0.1))
  layer1_biases = tf.Variable(tf.zeros([num_channels]))
  
  # S2: maxpool kernel [1, 2, 2, 1]
  
  # C3: kernel [5, 5, 1, 16]
  layer3_weights = tf.Variable(tf.truncated_normal([patch_size, patch_size, num_channels, depth], stddev=0.1))
  layer3_biases = tf.Variable(tf.zeros([depth]))
  
  # S4: maxpool kernel [1, 2, 2, 1]
  
  # C5: kernel [1, 5, 5, 1]
  layer5_weights = tf.Variable(tf.truncated_normal([patch_size, patch_size, depth, num_hidden_c5], stddev=0.1))
  layer5_biases = tf.Variable(tf.zeros([num_hidden_c5]))
  
  # F6: 
  layer6_weights = tf.Variable(tf.truncated_normal([num_hidden_c5, num_hidden], stddev=0.1))
  layer6_biases = tf.Variable(tf.constant(1.0, shape=[num_hidden]))
  
  # Output
  output_weights = tf.Variable(tf.truncated_normal([num_hidden, num_labels], stddev=0.1))
  output_biases = tf.Variable(tf.constant(1.0, shape=[num_labels]))
  
  # Model.
  def model(data):
    # C1 & S2
    conv = tf.nn.conv2d(data, layer1_weights, [1, 1, 1, 1], padding='SAME')
    pool1 = tf.nn.max_pool(conv, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='SAME', name='pool1')
    hidden = tf.nn.relu(pool1 + layer1_biases)
    # print("S2 shape: ", hidden[0], hidden[1], hidden[2], hidden[3])
    
    # C3 & S4
    conv = tf.nn.conv2d(hidden, layer3_weights, [1, 1, 1, 1], padding='VALID')
    pool2 = tf.nn.max_pool(conv, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='SAME', name='pool2')
    hidden = tf.nn.relu(pool2 + layer3_biases)
    # print("S4 shape: ", hidden[0], hidden[1], hidden[2], hidden[3])
    
    # C5
    conv = tf.nn.conv2d(hidden, layer5_weights, [1, 1, 1, 1], padding='VALID')
    hidden = tf.nn.relu(conv + layer5_biases)
    shape = hidden.get_shape().as_list()
    c5 = tf.reshape(hidden, [shape[0], shape[1] * shape[2] * shape[3]])
    # print("C5 shape: ", shape[0], shape[1], shape[2], shape[3])
    
    # F6
    # fc = tf.contrib.layers.fully_connected(c5, num_hidden, activation_fn=tf.sigmoid)
    fc = tf.nn.relu(tf.matmul(c5, layer6_weights) + layer6_biases)
    # shape = c6.get_shape().as_list()
    # fc = tf.reshape(c6, [shape[0], shape[1] * shape[2] * shape[3]])
    # print("F6 shape: ", shape[0], shape[1], shape[2], shape[3])
    
    drop = tf.nn.dropout(fc, keep_prob)
    
    # Output
    return tf.matmul(drop, output_weights) + output_biases
  
  # Training computation.
  logits = model(tf_train_dataset)
  loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels, logits=logits))
    
  # Optimizer.
  optimizer = tf.train.GradientDescentOptimizer(0.05).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits)
  valid_prediction = tf.nn.softmax(model(tf_valid_dataset))
  test_prediction = tf.nn.softmax(model(tf_test_dataset))

In [18]:
num_steps = 3001

with tf.Session(graph=graph) as session:
  tf.global_variables_initializer().run()
  print('Initialized')
  for step in range(num_steps):
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    batch_data = train_dataset[offset:(offset + batch_size), :, :, :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels, keep_prob: 1.0}
    feed_dict_w_drop = {tf_train_dataset : batch_data, tf_train_labels : batch_labels, keep_prob: 0.5}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict_w_drop)
    if (step % 100 == 0):
      print('Minibatch loss at step %d: %f' % (step, l))
      print('Minibatch accuracy: %.1f%%' % accuracy(predictions, batch_labels))
      print('Validation accuracy: %.1f%%' % accuracy(
        valid_prediction.eval(feed_dict=feed_dict), valid_labels))
  print('Test accuracy: %.1f%%' % accuracy(test_prediction.eval(feed_dict=feed_dict), test_labels))

Initialized
Minibatch loss at step 0: 2.976633
Minibatch accuracy: 6.2%
Validation accuracy: 10.1%
Minibatch loss at step 100: 1.484421
Minibatch accuracy: 56.2%
Validation accuracy: 60.6%
Minibatch loss at step 200: 1.316667
Minibatch accuracy: 50.0%
Validation accuracy: 72.3%
Minibatch loss at step 300: 1.271112
Minibatch accuracy: 56.2%
Validation accuracy: 72.5%
Minibatch loss at step 400: 1.005213
Minibatch accuracy: 68.8%
Validation accuracy: 78.0%
Minibatch loss at step 500: 1.155666
Minibatch accuracy: 68.8%
Validation accuracy: 77.8%
Minibatch loss at step 600: 1.130422
Minibatch accuracy: 68.8%
Validation accuracy: 78.0%
Minibatch loss at step 700: 0.331377
Minibatch accuracy: 93.8%
Validation accuracy: 80.4%
Minibatch loss at step 800: 1.220455
Minibatch accuracy: 75.0%
Validation accuracy: 79.3%
Minibatch loss at step 900: 0.685580
Minibatch accuracy: 81.2%
Validation accuracy: 80.9%
Minibatch loss at step 1000: 1.103068
Minibatch accuracy: 75.0%
Validation accuracy: 81.5%
